In [1]:
import numpy as np
import json
import pandas as pd
from tqdm import trange
import argparse
from modules.gem import GeM
# from modules.swin.swin_transformer import SwinTransformer
from modules.swin.build import build_model
from modules.swin.config import get_config
from modules.gswin import SwinFM
from utils.train_util import set_seed
from torch.utils.data import DataLoader
from datasets.dl import GeMData, GeMClass
from datasets.config import GeMConfig, SwinConfig
import torch
import os
from tqdm import tqdm
# import yaml
import gc
import torch.nn.functional as F
from matplotlib import pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
from sklearn.metrics import roc_auc_score, accuracy_score
# InteractiveShell.ast_node_interactivity = "all"

In [2]:
mi = GeMConfig('cifar100')
mi.set_arch('swin')
model = GeM(mi)

=> merge config from ./swin_para/swin_large_patch4_window7_224_22k.yaml


/home/azon/anaconda3/envs/coen344/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180507909/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


==============> Loading weight ./swin_para/swin_large_patch4_window7_224_22k.pth for fine-tuning......
loading ImageNet-22K weight to ImageNet-1K ......
_IncompatibleKeys(missing_keys=['layers.0.blocks.0.attn.relative_position_index', 'layers.0.blocks.1.attn_mask', 'layers.0.blocks.1.attn.relative_position_index', 'layers.1.blocks.0.attn.relative_position_index', 'layers.1.blocks.1.attn_mask', 'layers.1.blocks.1.attn.relative_position_index', 'layers.2.blocks.0.attn.relative_position_index', 'layers.2.blocks.1.attn_mask', 'layers.2.blocks.1.attn.relative_position_index', 'layers.2.blocks.2.attn.relative_position_index', 'layers.2.blocks.3.attn_mask', 'layers.2.blocks.3.attn.relative_position_index', 'layers.2.blocks.4.attn.relative_position_index', 'layers.2.blocks.5.attn_mask', 'layers.2.blocks.5.attn.relative_position_index', 'layers.2.blocks.6.attn.relative_position_index', 'layers.2.blocks.7.attn_mask', 'layers.2.blocks.7.attn.relative_position_index', 'layers.2.blocks.8.attn.relat

In [3]:
# pic_matrix = torch.ByteTensor(np.load("data/imageset_small.npy"))
pic_matrix = torch.ByteTensor(np.random.randint(low=0, high=255, size=(256, 3, 224, 224)))
dataset_img = torch.LongTensor(np.random.randint(low=0, high=256, size=(1280, 1)))
dataset_label = torch.LongTensor(np.random.randint(low=0, high=100, size=(1280, 1)))

In [4]:
train_dataset = GeMClass(pic_matrix, torch.cat([dataset_img, dataset_label], dim=-1))
train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# input_label = torch.zeros((32), dtype=torch.long).to(0)
input_label = torch.randint(low=0, high=100, size=(32, 1)).reshape(-1)

In [5]:
optimizer = torch.optim.AdamW(params=model.parameters(), lr=5e-4)

In [7]:
for epoch in range(2):
    model.train()
    model.zero_grad()
    index = 0
    steps_one_epoch = len(train_data_loader)
    enum_dataloader = tqdm(train_data_loader, total=steps_one_epoch, desc="EP-{} train".format(epoch))
    loss_list = []
    for data in enum_dataloader:
    #     if index >= steps_one_epoch:
    #         break

        model_in = data[:, :-1] / 255.0
#         pred = model.st(model_in.reshape(-1, 3, 224, 224))
#         print(pred.size())
        pred = model.predict_class(model_in, 224)
        loss = F.cross_entropy(pred, data[:, -1])
        loss_list.append(loss)

        loss.backward()
        optimizer.step()
        model.zero_grad()

        enum_dataloader.set_description("EP-{} train loss: {}".format(epoch, loss))
        enum_dataloader.refresh()
        index += 1
    
    print('epoch {} end'.format(epoch))


EP-0 train:   0%|                                                                                | 0/40 [00:00<?, ?it/s]

torch.Size([32, 4, 1536]) torch.Size([32, 4]) torch.Size([32, 4, 1536])


/home/azon/anaconda3/envs/coen344/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
EP-0 train loss: 4.607869625091553:   2%|█▏                                              | 1/40 [01:04<41:36, 64.02s/it]


KeyboardInterrupt: 